# Cytoscape.js to Sigma.js 

## What's this?
Script to generate sigma.js file from Cytoscape.js JSON.



In [5]:
import json
import matplotlib.pyplot as plt

Blues = plt.get_cmap('Blues')

TAG_SCORE = 'Similarity_Score'

In [6]:
def find_min_max(nodes):
    scores = []
    
    for node in nodes:
         if TAG_SCORE in node["data"]:
                scores.append(node["data"][TAG_SCORE])
    
    min_score = min(scores)
    max_score = max(scores)
    
    return (min_score, max_score)

In [7]:
def convert2sigma(in_file_name, out_file_name):
    print('Source file: ' + in_file_name)

    # Load Original Cyjs file
    cyjs_file = open(in_file_name)
    source_json = json.load(cyjs_file)
    cyjs_file.close()
    
    # Output files
    out = open(out_file_name, "w")

    nodeMap = {}

    sourceNodes = source_json["elements"]["nodes"]
    sourceEdges = source_json["elements"]["edges"]

    min_max = find_min_max(sourceNodes)
    score_range = min_max[1] - min_max[0]
    
    nodeList = []
    edgeList = []

    for node in sourceNodes:
        
        node_name = node["data"]["name"]
        
        if node['data']['Curated_name'] is '':
            termID = node["data"]["name"]
        else:
            termID = node["data"]["Curated_name"]
           
        x = node["position"]["x"]/10
        y = node["position"]["y"]/10

        suid = node["data"]["SUID"]
        if "Num_Genes" in node["data"]:
            size = node["data"]["Num_Genes"] * 50
        else:
            size = 50.0
        
        if TAG_SCORE in node["data"]:
            ratio = (node["data"][TAG_SCORE] - min_max[0]) / score_range
            col1 = Blues(ratio)
            color = (
                int(col1[0] * 255),
                int(col1[1] * 255),
                int(col1[2]* 255)
            )
            print(str(color))
        else:
            color = (200,200,200)      
        
        color_str = 'rgb' + str(color)
        newNode = {
            'id':node_name, 
            'label':termID,
            'size':size,
            'x':x, 'y':y, 
            'color': color_str
        }
        
        nodeMap[str(suid)] = node_name
        nodeList.append(newNode)

    for edge in sourceEdges:
        source = nodeMap[edge['data']['source']]
        target = nodeMap[edge['data']['target']]
        interaction = edge['data']['interaction']
        edgeList.append({'source':source, 'target':target, 'relationship':interaction, 'weight':1.0})

    graph = {}

    graph["nodes"] = nodeList
    graph["edges"] = edgeList

    out.write(json.dumps(graph, sort_keys=True, indent=4))
    out.close()

In [8]:
# Test run
convert2sigma('./data/8_22_2016/atgo2_tree.cyjs', './data/8_22_2016/atgo.json')

Source file: ./data/8_22_2016/atgo2_tree.cyjs
(206, 224, 241)
(117, 179, 216)
(151, 198, 223)
(209, 226, 242)
(208, 225, 242)
(162, 203, 226)
(166, 205, 227)
(209, 226, 242)
(18, 94, 166)
(192, 216, 237)
(247, 251, 255)
(160, 202, 225)
(115, 178, 215)
(163, 204, 227)
(162, 203, 226)
(205, 223, 241)
(58, 138, 193)
(89, 162, 207)
(15, 91, 163)
(130, 186, 219)
(207, 225, 242)
(160, 202, 225)
(87, 160, 206)
(155, 200, 224)
(161, 203, 226)
(209, 226, 242)
(33, 113, 181)
(63, 143, 196)
(166, 205, 227)
(40, 120, 184)
(47, 127, 188)
(8, 73, 145)
(84, 158, 205)
(65, 145, 197)
(45, 125, 187)
(32, 112, 180)
(8, 66, 134)
(90, 163, 207)
(8, 70, 140)
(8, 71, 142)
(15, 90, 163)
(8, 80, 154)
(8, 80, 154)
(28, 107, 176)
(101, 170, 211)
(166, 205, 227)
(207, 225, 242)
(14, 89, 162)
(14, 89, 162)
(24, 102, 172)
(19, 96, 167)
(18, 94, 166)
(14, 89, 162)
(13, 88, 161)
(35, 115, 182)
(93, 164, 208)
(80, 155, 203)
(178, 210, 232)
(8, 58, 122)
(112, 177, 215)
(40, 120, 184)
(64, 144, 197)
(65, 145, 197)
(87, 